<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/ChatGPT_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**LangChain APP**

https://qiita.com/wwwcojp/items/c7f43c5f964b8db8a890

https://note.com/npaka/n/n6b7a07e492f1

In [3]:
# パッケージのインストール
!pip install langchain --q
!pip install openai --q
!pip install google-search-results --q
!pip install --upgrade deepl --q
import openai
import deepl
import textwrap
import pandas as pd

# パッケージのインポート
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
with open("/content/drive/MyDrive/Deep_learning/api.txt") as file:
    #text = file.read()
    i=1
    key = []
    while True:
        include_break_line = file.readline() #改行が含まれた行
        line = include_break_line.rstrip() #改行を取り除く
        if line: #keyの読み込み
            #print(f'{i}行目：{line}')
            key.append(line)
            i += 1
        else:
            break

# APIキーの準備
# #ngrok_aceess_token = key[5]
# openai.api_key = key[3]
# deepl_auth_key = key[1]
# serp_api_key = key[7]

import os
os.environ["OPENAI_API_KEY"] = key[3]
os.environ["SERPAPI_API_KEY"] = key[7]

In [8]:
# LLMの準備準備
llm = OpenAI(temperature=0)

# ツールの準備
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# エージェントの準備
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True
)

In [10]:
# エージェントの実行
agent.run("おでこにヒアルロン酸を注入する際の合併症は？")



> Entering new AgentExecutor chain...
 I need to find out what the complications of injecting hyaluronic acid into the forehead are.
Action: Search
Action Input: Hyaluronic acid forehead injection complications
Observation: Fortunately, over 90% of adverse events from the use of HA fillers are mild and transient. These include injection site redness, swelling, or bruising. However, disastrous outcomes can occur, including necrosis, vision loss, and cerebrovascular accidents.
Thought: I now know the potential complications of injecting hyaluronic acid into the forehead.
Final Answer: The potential complications of injecting hyaluronic acid into the forehead include injection site redness, swelling, bruising, necrosis, vision loss, and cerebrovascular accidents.

> Finished chain.


'The potential complications of injecting hyaluronic acid into the forehead include injection site redness, swelling, bruising, necrosis, vision loss, and cerebrovascular accidents.'

#**例を見ながら勉強**

## 基本的な入出力

In [29]:
from langchain.llms import OpenAI

# モデル名を指定しない場合、「text-davinci-003」が利用されます
llm = OpenAI(temperature=0.9)
text = "国立大学法人東京工業大学と国立大学法人東京医科歯科大学が合併しました。ついクスっと笑ってしまう新名称を考えてください。"
print(llm(prompt=text))



「東京テクノロジー医科歯科大学」


## PromptTemplate

ユーザー入力や複数出力に対応

In [15]:
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["target"],
    template="技術記事のタイトルをよしなに生成してくれるAIの{target}を考えてください。",
)
print(llm(prompt=prompt.format(target="セールスポイント")))



・最新のテクノロジー分野で有効な技術に関する記事タイトルを自動生成するAIを提供します。

・最新の研究成果を内部で解析して、記事タイトルに適したワードを抽出し、効率的な記事タイトルを提供します。

・記事タイトルの作成にお金や時間を節約します。

・テクノロジーを専門とする専門家がいなくても、より良質な記事タイトルをつくることができます。

・新しい技


## Chains

指定した順番で作業を行う

In [20]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=.7)
template = """あなたは新商品の開発を担当しています。新商品のターゲットが与えられたら、ターゲットに売れる新商品のアイデアを出してください。

ターゲット: {target}
担当者: 以下が新商品のアイデアです。"""
prompt_template = PromptTemplate(input_variables=["target"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

llm = OpenAI(temperature=.7)
template = """あなたは経営者です。新商品のアイデアが与えられたら、経営者の観点から批判的にレビューをしてください。

アイデア:
{idea}
経営者：
"""
prompt_template = PromptTemplate(input_variables=["idea"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)
review = overall_chain.run("通勤に1時間以上かかっている20代の男性")



> Entering new SimpleSequentialChain chain...


1.高性能のノートPC: 20代の男性にとって、ノートPCは非常に便利なツールです。ノートPCは、高性能なパフォーマンスを提供しながら、コンパクトで軽量なので、通勤中の仕事や勉強などに最適です。

2. 無線LAN・モバイルルーター: 通勤中の電車内でもインターネット接続ができる無線LAN・モバイルルーターは、20代の男性にとって便利な商品です。インターネット接続ができるので、電車内でも仕事や勉強を続

1.高性能のノートPC: 確かに、20代の男性にとって高性能のノートPCは非常に便利なツールです。しかし、値段が高くなりすぎると、市場から受け入れられない可能性があります。また、他のノートPCとの差別化を図るため、価値を高めるための新しい機能を提供する必要があるでしょう。

2.無線LAN・モバイルルーター: 確かに、20代の男性にとって便利な商品ですが、他の製品との差別化を行う

> Finished chain.


## Agents

Chainをどのような順番で動かすかを決定する

・zero-shot-react-description：  Toolのdescription（説明文）をもとに、どのToolを選択するか決定する

・react-docstore:  ドキュメントを検索するSearchと、検索結果から用語を探すLookupという2つのToolを用意

・self-ask-with-search:  Google検索等を用いて、自問自答・検索を繰り返しながら答えを導く





In [30]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

llm = OpenAI(temperature=0)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
agent.run("日本で一番高い山の高さは何メートルですか?その高さの5乗根を求めてください。")



> Entering new AgentExecutor chain...
 I need to find the height of the highest mountain in Japan and then calculate its fifth root.
Action: Search
Action Input: "highest mountain in Japan"
Observation: Mount Fuji
Thought: I need to find the height of Mount Fuji
Action: Search
Action Input: "height of Mount Fuji"
Observation: 12,388′
Thought: I need to convert this to meters
Action: Calculator
Action Input: 12,388 * 0.3048

> Entering new LLMMathChain chain...
12,388 * 0.3048
Answer: 3760.544
> Finished chain.

Observation: Answer: 3760.544
Thought: I need to calculate the fifth root of 3760.544
Action: Calculator
Action Input: 3760.544^(1/5)

> Entering new LLMMathChain chain...
3760.544^(1/5)
```python
print(3760.544**(1/5))
```

Answer: 5.188599325914775

> Finished chain.

Observation: Answer: 5.188599325914775

Thought: I now know the final answer
Final Answer: The height of the highest mountain in Japan is 3760.544 meters and its fifth root is 5.188599325914775.

> Finished cha

'The height of the highest mountain in Japan is 3760.544 meters and its fifth root is 5.188599325914775.'

## Memory

前までの会話を覚えておく

・  ConversationBufferMemory  前段までの入力・出力をそのまま記憶する

・  ConversationalBufferWindowMemory  k段前までの入力・出力をそのまま記憶する

・  ConversationSummaryMemory  前段までの入力・出力をLLMでサマライズして記憶する